In [1]:
from pathlib import Path


In [2]:
path_to_csv = '/home/ttnlsc/Документы/python_balstim/files_for_courses'
path_to_csv = Path(path_to_csv)

In [11]:
file = open(path_to_csv / '100723.csv')

data = file.read()

file.close()

In [8]:
clinical_significance = {'Pathogenic':' как патогенный клинически значимый вариант',
                         'Pathogenic/Likely pathogenic':' как патогенный клинически значимый вариант',
                         'Likely Pathogenic':' как вероятно патогенный клинически значимый вариант',
                         'Likely pathogenic':' как вероятно патогенный клинически значимый вариант', 
                         'Uncertain significance':' как вариант с неизвестным клиническим значением', 
                         'Conflicting interpretations of pathogenicity. risk factor':', имеет конфликтующие интерпретации патогенности',
                         'Conflicting interpretations of pathogenicity':', имеет конфликтующие интерпретации патогенности',
                         'not provided':'.',
                        '':' не зарегистрирована'}
acmg_significance = {'5':'патогенным вариантом', '4':'вероятно патогенным вариантом', '3':'вариантом с неизвестным клиническим значением'}
sequence_ontology = {'missense_variant':'замене аминокислоты', 
                     'frameshift_truncation':'сдвигу рамки считывания и преждевременной остановке синтеза белка',
                     'frameshift_elongation':'сдвигу рамки считывания',
                     'stop_gained':'преждевременной остановке синтеза белка',
                     'inframe_insertion':'вставке без сдвига рамки считывания'}
transcripts = {'BRCA1':'NM_007294.4', 'BRCA2':'NM_000059.4', 'ATM':'NM_000051.4', 'CHEK2':'NM_007194.4', 'PALB2':'NM_024675.4', 'RAD51D':'NM_002878.4', 'BRIP1':'NM_032043.3'}

In [12]:
splited_data = data.splitlines()
header = splited_data[0].split(',')
splited_data = splited_data[1:]

result = header.index('Результат')
number = header.index('№')
material = header.index('материал')
reads = header.index('PF_READS')
bases = header.index('PF_BASES')
cover = header.index('MEAN COVER')
numffpe = header.index('num_ffpe')
gene = header.index('Gene')
chrom = header.index('Chrom')
pos = header.index('Position')
ref = header.index('Ref Base')
alt = header.index('Alt Base')
coding = header.index('cDNA change')
protein = header.index('Protein Change')
effect = header.index('Sequence Ontology')
clinvar = header.index('Clinical Significance')
acmg = header.index('Класс ACMG')
depth = header.index('Total reads')
freq = header.index('Global AF')
AB = header.index('Variant AF')
rs = header.index('rsID')


In [13]:
for line in splited_data:
    line = line.split(',')
    name = line[number].split('/')[0]
    dest_file = open(f'{path_to_csv / name}.txt', mode = 'w')
    dest_str = 'Выполнен анализ наличия мутаций в кодирующих и прилегающих областях генов BRCA1/BRCA2 (RefSeq: NCBI Reference Sequence Database - NM_007294.4, NM_000059.4).\n'
    dest_str += 'Исследование проводилось с использованием набора реагентов "Соло-тест АВС плюс" (РУ № РЗН 2023/20034) на приборе Illumina MiSeq методом высокопроизводительного секвенирования по ТУ 21.20.23-005-91709359-2022.\n'
    dest_str += f'Диагностические характеристики запуска:\nКоличество прочтений на образец - {line[reads]}\nПрочитано оснований - {line[bases]}\nСреднее значение покрытия образца - {round(float(line[cover]), 1)} \n'
    class_a = line[acmg]
    tr_gene = line[gene]
    clinsig = line[clinvar]
    seq_ont = line[effect]

    if line[result] == 'wt':
        if line[material] == 'кровь':
            dest_str += 'При исследовании ДНК, выделенной из лейкоцитов периферической крови, патогенных клинически значимых мутаций в генах BRCA1/BRCA2 не выявлено.'
        elif line[material] == 'блок':
            dest_str += f'Исследованный материал: блок № {line[numffpe]}\nПри исследовании ДНК, выделенной из фиксированной в формалине и залитой парафином ткани, патогенных клинически значимых мутаций в генах BRCA1/BRCA2 не выявлено.'
        dest_file.write(dest_str)
        dest_file.close()
    elif line[result] == 'mut':
        if line[material] == 'кровь':
            if line[gene] == 'BRCA1' or line[gene] == 'BRCA2':
                dest_str += f'При исследовании ДНК, выделенной из лейкоцитов периферической крови, в гене {line[gene]} ({transcripts[tr_gene]}) выявлена герминальная мутация {line[coding]} (hg38 {line[chrom]}:{line[pos]}-{line[ref]}-{line[alt]}, {line[protein]}, {line[rs]}) в гетерозиготном состоянии, приводящая к {sequence_ontology[seq_ont]}.\n'
                if line[freq] != '':
                    dest_str += f'Частота минорного аллеля (gnomAD_genomes v3) {float(line[freq]) * 100}%.\n'
                else:
                    dest_str += 'Вариант отсутствует в базе данных популяционных частот gnomAD_genomes v3.\n'
                dest_str += f'Глубина прочтения варианта {line[depth]}x.\n'
                dest_str += f'Мутация {line[coding]} в гене {line[gene]} зарегистрирована в международных базах данных dbSNP, ClinVar, BRCA Exchange{clinical_significance[clinsig]}.\nСогласно ACMG классификации мутация {line[coding]} в гене {line[gene]} является {acmg_significance[class_a]}, класс мутаций {line[acmg]}.\nРекомендована консультация врача-генетика.'
                dest_file.write(dest_str)
                dest_file.close()
            else:
                dest_str += 'При исследовании ДНК, выделенной из лейкоцитов периферической крови, патогенных клинически значимых мутаций в генах BRCA1/BRCA2 не выявлено.\n'
                dest_str += f'Примечание: в гене {line[gene]} ({transcripts[tr_gene]}) выявлена герминальная мутация {line[coding]} (hg38 {line[chrom]}:{line[pos]}-{line[ref]}-{line[alt]}, {line[protein]}, {line[rs]}) в гетерозиготном состоянии, приводящая к {sequence_ontology[seq_ont]}.\n'
                if line[freq] != '':
                    dest_str += f'Частота минорного аллеля (gnomAD_genomes v3) {float(line[freq]) * 100}%.\n'
                else:
                    dest_str += 'Вариант отсутствует в базе данных популяционных частот gnomAD_genomes v3.\n'
                dest_str += f'Глубина прочтения варианта {line[depth]}x.\n'
                dest_str += f'Мутация {line[coding]} в гене {line[gene]} зарегистрирована в международных базах данных dbSNP, ClinVar{clinical_significance[clinsig]}.\nСогласно ACMG классификации мутация {line[coding]} в гене {line[gene]} является {acmg_significance[class_a]}, класс мутаций {line[acmg]}.\nРекомендована консультация врача-генетика.'
                if line[coding] == 'c.470T>C' and line[gene] == 'CHEK2':
                    dest_str += '\nСогласно клиническим рекомендациям NCCN, изменение тактики ведения для пациентов носителей данного варианта не требуется.'
                dest_file.write(dest_str)
                dest_file.close()
        elif line[material] == 'блок':
            if line[gene] == 'BRCA1' or line[gene] == 'BRCA2':
                dest_str += f'Исследованный материал: блок № {line[numffpe]}\nПри исследовании ДНК, выделенной из фиксированной в формалине и залитой парафином ткани, в гене {line[gene]} ({transcripts[tr_gene]}) выявлена мутация {line[coding]} (hg38 {line[chrom]}:{line[pos]}-{line[ref]}-{line[alt]}, {line[protein]}, {line[rs]}), аллельный баланс {round(float(line[AB]), 2)}, приводящая к {sequence_ontology[seq_ont]}.\n'
                if line[freq] != '':
                    dest_str += f'Частота минорного аллеля (gnomAD_genomes v3) {float(line[freq]) * 100}%.\n'
                else:
                    dest_str += 'Вариант отсутствует в базе данных популяционных частот gnomAD_genomes v3.\n'
                dest_str += f'Глубина прочтения варианта {line[depth]}x.\n'
                dest_str += f'Мутация {line[coding]} в гене {line[gene]} зарегистрирована в международных базах данных dbSNP, ClinVar, BRCA Exchange{clinical_significance[clinsig]}.\nСогласно ACMG классификации мутация {line[coding]} в гене {line[gene]} является {acmg_significance[class_a]}, класс мутаций {line[acmg]}.\nКласс по АМР - \nРекомендована консультация врача-генетика.'
                dest_file.write(dest_str)
                dest_file.close()
            else:
                dest_str += f'Исследованный материал: блок № {line[numffpe]}\nПри исследовании ДНК, выделенной из фиксированной в формалине и залитой парафином ткани, патогенных клинически значимых мутаций в генах BRCA1/BRCA2 не выявлено.\n'
                dest_str += f'Примечание: в гене {line[gene]} ({transcripts[tr_gene]}) выявлена мутация {line[coding]} (hg38 {line[chrom]}:{line[pos]}-{line[ref]}-{line[alt]}, {line[protein]}, {line[rs]}), аллельный баланс {round(float(line[AB]), 2)}, приводящая к {sequence_ontology[seq_ont]}.\n'
                if line[freq] != '':
                    dest_str += f'Частота минорного аллеля (gnomAD_genomes v3) {float(line[freq]) * 100}%.\n'
                else:
                    dest_str += 'Вариант отсутствует в базе данных популяционных частот gnomAD_genomes v3.\n'
                dest_str += f'Глубина прочтения варианта {line[depth]}x.\n'
                dest_str += f'Мутация {line[coding]} в гене {line[gene]} зарегистрирована в международных базах данных dbSNP, ClinVar{clinical_significance[clinsig]}.\nСогласно ACMG классификации мутация {line[coding]} в гене {line[gene]} является {acmg_significance[class_a]}, класс мутаций {line[acmg]}.\nКласс по АМР - \nРекомендована консультация врача-генетика.'
                if line[coding] == 'c.470T>C' and line[gene] == 'CHEK2':
                    dest_str += '\nСогласно клиническим рекомендациям NCCN, изменение тактики ведения для пациентов носителей данного варианта не требуется.'
                dest_file.write(dest_str)
                dest_file.close()